In [ ]:
import pandas as pd
import polars as pl
import numpy  as np
import networkx as nx
from math import pi, cos, sin, sqrt
import random
from os.path import exists
from rtsvg import *
rt = RACETrack()
edges_filename  = '../../data/stanford/facebook/348.edges'
layout_filename = '../../data/stanford/facebook/348.edges.layout.parquet'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
my_df  = pl.DataFrame(_lu_)

In [ ]:
class SingleNodeFocus(object):
    #
    #
    #
    def __init__(self, rt_self, df, relationships, node_focus=None, count_by=None, count_by_set=False, color_by=None, txt_h=12, w=384, h=384):
        self.rt_self       = rt_self
        self.df            = df    
        self.relationships = relationships
        self.count_by      = count_by
        self.count_by_set  = count_by_set
        self.color_by      = color_by
        self.txt_h         = txt_h
        self.w             = w
        self.h             = h
        self.last_render   = None
        self.g             = self.rt_self.createNetworkXGraph(self.df, self.relationships)
        self.node_focus    = node_focus
        if self.node_focus is None: self.node_focus = list(self.g.nodes())[0]
        self.cx, self.cy   = self.w/2, self.h/2
    
    #
    #
    #
    def _repr_svg_(self):
        if self.last_render is None: self.renderSVG()
        return self.last_render

    #
    #
    #     
    def __renderImmediateNeighbors__(self):
        svg = []
        _fm_, _to_ = self.relationships[0][0], self.relationships[0][1]
        _set_       = set([self.node_focus])
        _df_nbors_  = self.df.filter(pl.col(_fm_).is_in(_set_) | pl.col(_to_).is_in(_set_))
        _set_nbors_ = (set(_df_nbors_['fm']) | set(_df_nbors_['to'])) - _set_
        if len(_set_nbors_) > 0:
            ...

        return ''.join(svg)

    #
    #
    #
    def renderSVG(self):
        self.pos = {self.node_focus: (self.cx, self.cy)}
        svg = [f'<svg x="0" y="0" width="{self.w}" height="{self.h}">']
        svg.append(f'<rect width="{self.w}" height="{self.h}" x="0" y="0" fill="{self.rt_self.co_mgr.getTVColor('background','default')}" />')
        svg.append(self.__renderImmediateNeighbors__())
        svg.append(f'<circle cx="{self.cx}" cy="{self.cy}" r="5" stroke="black" stroke-width="2" fill="{self.rt_self.co_mgr.getTVColor('data','default')}" />')
        svg.append(self.rt_self.svgText(f"{self.node_focus}", self.cx, self.cy + self.txt_h + 4, 'black', anchor='middle'))
        self.last_render = ''.join(svg) + '</svg>'

#
_snf_ = SingleNodeFocus(rt, my_df, [('fm','to')])
rt.tile([_snf_])